# Homework 3

In [23]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [24]:
df = pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [55]:
df.head()

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,7590-vhveg,female,0,yes,no,1,no,no_phone_service,dsl,no,...,no,no,no,no,month-to-month,yes,electronic_check,29.85,29.85,0
1,5575-gnvde,male,0,no,no,34,yes,no,dsl,yes,...,yes,no,no,no,one_year,no,mailed_check,56.95,1889.50,0
2,3668-qpybk,male,0,no,no,2,yes,no,dsl,yes,...,no,no,no,no,month-to-month,yes,mailed_check,53.85,108.15,1
3,7795-cfocw,male,0,no,no,45,no,no_phone_service,dsl,yes,...,yes,yes,no,no,one_year,no,bank_transfer_(automatic),42.30,1840.75,0
4,9237-hqitu,female,0,no,no,2,yes,no,fiber_optic,no,...,no,no,no,no,month-to-month,yes,electronic_check,70.70,151.65,1


In [33]:
df.columns = df.columns.str.lower().str.replace(' ','_')
df.head().T

,0,1,2,3,4
customerid,7590-vhveg,5575-gnvde,3668-qpybk,7795-cfocw,9237-hqitu
gender,female,male,male,male,female
seniorcitizen,0,0,0,0,0
partner,yes,no,no,no,no
dependents,no,no,no,no,no
tenure,1,34,2,45,2
phoneservice,no,yes,yes,no,yes
multiplelines,no_phone_service,no,no,no_phone_service,no
internetservice,dsl,dsl,dsl,dsl,fiber_optic
onlinesecurity,no,yes,yes,yes,no


In [27]:
categorical_columns = list(df.dtypes[df.dtypes=='object'].index)

In [82]:
df.dtypes

customerid           object
gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure                int64
phoneservice         object
multiplelines        object
internetservice      object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv          object
streamingmovies      object
contract             object
paperlessbilling     object
paymentmethod        object
monthlycharges      float64
totalcharges        float64
churn                 int64
risk                float64
dtype: object

In [32]:
for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ','_')

In [47]:
df.totalcharges = pd.to_numeric(df.totalcharges, errors = 'coerce')

In [54]:
df.churn = (df.churn == 'yes').astype(int)

In [49]:
df.totalcharges = df.totalcharges.fillna(0)

## prepare data

In [59]:
from sklearn.model_selection import train_test_split

In [60]:
full_train ,test_set = train_test_split(df, test_size =0.2, random_state= 1)
train_set ,val_set = train_test_split(full_train, test_size =0.25, random_state= 1)

In [61]:
train_set = train_set.reset_index(drop=True)
val_set = val_set.reset_index(drop=True)
test_set = test_set.reset_index(drop=True)

In [65]:
y_train = train_set['churn']
y_val = val_set['churn']
y_test = test_set['churn']

In [66]:
del train_set['churn']
del val_set['churn']
del test_set['churn']

## EDA

In [88]:
from IPython.display import display

In [104]:
categorical = [
    'gender',
    'seniorcitizen',
    'partner',
    'dependents',
    'phoneservice',
    'multiplelines',
    'internetservice',
    'onlinesecurity',
    'onlinebackup',
    'deviceprotection',
    'techsupport',
    'streamingtv',
    'streamingmovies',
    'contract',
    'paperlessbilling',
    'paymentmethod',
]
numerical = ['tenure', 'monthlycharges', 'totalcharges']


In [92]:
df.churn.value_counts(normalize = True)

churn
0    0.73463
1    0.26537
Name: proportion, dtype: float64

In [93]:
full_train
global_churn = full_train.churn.mean()
global_churn

np.float64(0.26996805111821087)

In [95]:
for c in categorical:
    df_group = full_train.groupby(c).churn.agg(['count','mean'])
    df_group['risk'] = df_group['mean']/ global_churn
    display(df_group)
    print()

,count,mean,risk
gender,,,
female,2796,0.276824,1.025396
male,2838,0.263214,0.974980


,count,mean,risk
seniorcitizen,,,
0,4722,0.242270,0.897403
1,912,0.413377,1.531208


,count,mean,risk
partner,,,
no,2932,0.329809,1.221659
yes,2702,0.205033,0.759472


,count,mean,risk
dependents,,,
no,3968,0.313760,1.162212
yes,1666,0.165666,0.613651


,count,mean,risk
phoneservice,,,
no,547,0.241316,0.893870
yes,5087,0.273049,1.011412


...

,count,mean,risk
streamingtv,,,
no,2246,0.342832,1.269897
no_internet_service,1221,0.077805,0.288201
yes,2167,0.302723,1.121328


,count,mean,risk
streamingmovies,,,
no,2213,0.338906,1.255358
no_internet_service,1221,0.077805,0.288201
yes,2200,0.307273,1.138182


,count,mean,risk
contract,,,
month-to-month,3104,0.431701,1.599082
one_year,1186,0.120573,0.446621
two_year,1344,0.028274,0.104730


,count,mean,risk
paperlessbilling,,,
no,2313,0.172071,0.637375
yes,3321,0.338151,1.252560


,count,mean,risk
paymentmethod,,,
bank_transfer_(automatic),1219,0.168171,0.622928
credit_card_(automatic),1217,0.164339,0.608733
electronic_check,1893,0.455890,1.688682
mailed_check,1305,0.193870,0.718121


## Mutual Information

In [96]:
from sklearn.metrics import mutual_info_score

In [98]:
mutual_info_score(full_train['gender'],full_train['churn'])

0.0001174846211139946

In [101]:
def mutual_info_churn_score(series):
    return mutual_info_score(series, full_train.churn)

In [103]:
mi = full_train[categorical].apply(mutual_info_churn_score).sort_values(ascending=False)

contract            0.098320
onlinesecurity      0.063085
techsupport         0.061032
internetservice     0.055868
onlinebackup        0.046923
deviceprotection    0.043453
paymentmethod       0.043210
streamingtv         0.031853
streamingmovies     0.031581
paperlessbilling    0.017589
dependents          0.012346
partner             0.009968
seniorcitizen       0.009410
multiplelines       0.000857
phoneservice        0.000229
gender              0.000117
dtype: float64

## Correlation

In [105]:
full_train[numerical].corrwith(full_train.churn)

tenure           -0.351885
monthlycharges    0.196805
totalcharges     -0.196353
dtype: float64